## Create Events from Plug Data

In [50]:
import json
import joule
from joule.errors import ApiError

In [51]:
PLUG_STREAM = "/NILM Training/Homes/Donnal Cambridge/Smart Plugs/Keurig"
EVENT_STREAM = "/NILM Training/Homes/Donnal Cambridge/Events"
LOAD_NAME = "Keurig"
PLUG_NODE = "lambda"
EVENT_NODE = "lambda"
MIN_WATTS = 120
REMOVE_EXISTING = True

In [52]:
node = joule.api.get_node(EVENT_NODE)

try:
    event_stream = await node.event_stream_get(EVENT_STREAM)
except ApiError:
    folder = "/".join(EVENT_STREAM.split('/')[:-1])
    name = EVENT_STREAM.split('/')[-1]
    event_stream = joule.api.EventStream(name=name, event_fields = {
        "name": "string",
        "average_power": "numeric",
        "kwh": "numeric",
        "peak_power": "numeric",
        "duration": "numeric"
    })
    event_stream = await node.event_stream_create(event_stream, folder)
await node.close()

In [53]:
node = joule.api.get_node(EVENT_NODE)

if REMOVE_EXISTING:
    json_filter = json.dumps([[['name','is',LOAD_NAME],['peak_power','gt',100.5]]])
    await node.event_stream_remove(event_stream, json_filter=json_filter)
await node.close()

In [54]:
plug_node = joule.api.get_node(PLUG_NODE)
event_node = joule.api.get_node(EVENT_NODE)
pipe = await plug_node.data_read(PLUG_STREAM)
peak_power = None
start_row = None
power_acc = 0
run_ticks = 0
while not pipe.is_empty():
    data = await pipe.read(flatten=True)
    pipe.consume(len(data))
    for row in data:
        if row[3]>MIN_WATTS and start_row is None:
            start_row = row
            peak_power = row[3]
            power_acc = row[3]
            run_ticks = 1
        if start_row is not None:
            peak_power = max((peak_power, row[3]))
            power_acc += row[3]
            run_ticks += 1
        if start_row is not None and row[3]<MIN_WATTS:
            event = joule.api.Event(start_row[0],row[0])
            event.content = {
                'average_power': power_acc/run_ticks,
                'kwh': row[7]-start_row[7],
                'name': LOAD_NAME,
                'peak_power': peak_power,
                'duration': (row[0]-start_row[0])/1e6
            }
            await event_node.event_stream_write(event_stream, [event])
            print("added event")
            start_row = None
            peak_power = None
            power_acc = 0
            run_ticks = 0
    print(f"read {len(data)} rows")
await plug_node.close()
await event_node.close()

read 676 rows
read 20099 rows
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
read 25950 rows
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
read 18911 rows
read 37415 rows
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added event
added even